In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from helper.helper import make_input_fn
from helper.img_to_df import img_to_df
from convert_to_img import convert_1D_array_to_img


In [5]:
rgb_train = pd.read_csv('after.csv')
r_train = rgb_train['R']
g_train = rgb_train['G']
b_train = rgb_train['B']
df_train = pd.read_csv('before.csv')

# extracting the last 10% of sample data to be used as evaluation set
num_of_eval_samples = int(0.1 * rgb_train.size)
r_eval = r_train[-num_of_eval_samples:]
r_train = r_train.iloc[:-num_of_eval_samples]
g_eval = g_train[-num_of_eval_samples:]
g_train = g_train.iloc[:-num_of_eval_samples]
b_eval = b_train[-num_of_eval_samples:]
b_train = b_train.iloc[:-num_of_eval_samples]
df_eval = df_train[-num_of_eval_samples:]
df_train = df_train[:-num_of_eval_samples]

# populating the feature_columns
feature_columns = []
for feature_name in df_train.columns:
    feature_columns.append(tf.feature_column.numeric_column(
        feature_name, dtype=tf.float64))

# call the input_function that was returned to us to get a dataset object we can feed to the model
r_train_input_fn = make_input_fn(df_train, r_train)
g_train_input_fn = make_input_fn(df_train, g_train)
b_train_input_fn = make_input_fn(df_train, b_train)
eval_input_fn = make_input_fn(df_eval, r_eval, num_epochs=1, shuffle=False)

# creating linear estimator for each of R, G, B channel
r_linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns)
g_linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns)
b_linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# training models
r_linear_est.train(r_train_input_fn)
g_linear_est.train(g_train_input_fn)
b_linear_est.train(b_train_input_fn)

#r_pred_dicts = list(r_linear_est.predict(r_eval_input_fn))
#r_predicted_output = pd.Series([pred['predictions'][0] for pred in r_pred_dicts])


:tensorflow:global_step/sec: 745.771
INFO:tensorflow:loss = 0.00071704364, step = 9300 (0.134 sec)
INFO:tensorflow:global_step/sec: 582.663
INFO:tensorflow:loss = 0.0024716458, step = 9400 (0.172 sec)
INFO:tensorflow:global_step/sec: 507.926
INFO:tensorflow:loss = 0.000604526, step = 9500 (0.197 sec)
INFO:tensorflow:global_step/sec: 565.986
INFO:tensorflow:loss = 0.002939335, step = 9600 (0.178 sec)
INFO:tensorflow:global_step/sec: 572.807
INFO:tensorflow:loss = 0.0028490312, step = 9700 (0.173 sec)
INFO:tensorflow:global_step/sec: 579.48
INFO:tensorflow:loss = 0.00059251185, step = 9800 (0.173 sec)
INFO:tensorflow:global_step/sec: 567.058
INFO:tensorflow:loss = 0.0006488842, step = 9900 (0.176 sec)
INFO:tensorflow:global_step/sec: 612.997
INFO:tensorflow:loss = 0.0021929652, step = 10000 (0.162 sec)
INFO:tensorflow:global_step/sec: 789.964
INFO:tensorflow:loss = 0.0011950757, step = 10100 (0.127 sec)
INFO:tensorflow:global_step/sec: 737.351
INFO:tensorflow:loss = 0.0005345374, step = 

In [6]:
# Predict

test_img = Image.open('test2_before_thumbnail.jpg')

df_test = pd.DataFrame(np.row_stack(np.asarray(
    test_img) / 255.0), columns=['R', 'G', 'B'])

r_pred_input_fn = make_input_fn(
    df_test, df_test['R'], num_epochs=1, shuffle=False)

g_pred_input_fn = make_input_fn(
    df_test, df_test['G'], num_epochs=1, shuffle=False)

b_pred_input_fn = make_input_fn(
    df_test, df_test['B'], num_epochs=1, shuffle=False)

r_pred_dicts = list(r_linear_est.predict(input_fn=r_pred_input_fn))
g_pred_dicts = list(g_linear_est.predict(input_fn=g_pred_input_fn))
b_pred_dicts = list(b_linear_est.predict(input_fn=b_pred_input_fn))

r_pred_arr = [pred['predictions'][0] for pred in r_pred_dicts]
g_pred_arr = [pred['predictions'][0] for pred in g_pred_dicts]
b_pred_arr = [pred['predictions'][0] for pred in b_pred_dicts]

rgb_pred_arr = np.empty((len(r_pred_arr), 3))
for i in range(len(rgb_pred_arr)):
    rgb_pred_arr[i] = [r_pred_arr[i], g_pred_arr[i], b_pred_arr[i]]

rgb_pred_arr = rgb_pred_arr * 255

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\kamalakm\AppData\Local\Temp\tmpwb_36u47\model.ckpt-13440
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\kamalakm\AppData\Local\Temp\tmpha4pt01e\model.ckpt-13440
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\kamalakm\AppData\Local\Temp\tmp7te99trl\model.ckpt-13440
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [6]:
thumbnail_size = 128, 128
thumbnail_img = Image.open('test_before.jpg')
thumbnail_img.thumbnail(thumbnail_size)
thumbnail_img.save('test_before_thumbnail.jpg')

In [7]:
new_img = Image.new('RGB', test_img.size)
i, j = (0, 0)
img_arr = rgb_pred_arr.astype(int)
i,j = (0,0)
for idx in range(len(img_arr)):
    if (i == test_img.size[0]):
        i = 0
        j += 1

    r, g, b = img_arr[idx]

    new_img.putpixel((i, j), (r, g, b))
    i += 1

new_img.save('test2_after_pred.jpg')